In [ ]:
from bs4 import BeautifulSoup
import requests
import os.path
from htmldate import find_date
import pandas as pd
def scrape(url):
    try:
        response = requests.get(url)
    except Exception:
        return "DELETE"
    html_content = response.text
    soup = BeautifulSoup(html_content, 'lxml')  # You can also use 'html.parser'
    paragraphs = soup.find_all("p")
    textOut = ""
    timeStamp = ""
    try:
        timeStamp = find_date(url)
        year = timeStamp.split("-")[0]
        month = timeStamp.split("-")[1]
        day = timeStamp.split("-")[2]
    except Exception:
        year = "check"
        month = "check"
        day = "check"
    removeList = []
    captions = soup.find_all("figcaption")
    for c in captions:
        removeList.append(c.get_text().strip().replace("\n",""))
    authorBio = soup.find("div",{"class":"author-bio__bio"})
    if authorBio!=None:
        removeList.append(authorBio.get_text().strip().replace("\n",""))
    #remove = soup.find_all("p",{"class":"u-speakableText-dek c-contentHeader_description g-outer-spacing-top-small"})
    #captions = soup.find_all("span",{"class":"c-shortcodeImage_caption g-inner-spacing-right-small g-text-xxsmall"})
    removeList.append("More OpenAI news:")
    for block,p in enumerate(paragraphs):
        text = p.get_text().replace("\n","").strip()
        #print(text)
        if text !="" and ((text[0:4]=="More" and text[-1]==":" and block> len(paragraphs)-8) or (text[0:16]=="Please follow me" and block> len(paragraphs)-5)):
            break
        if text != "" and text[-1] in '!.?:”)“"' and text not in removeList: #this varies between websites since some write in a more academic manner
            textOut+= " " + text
    length = 0
    if textOut == "":
        return "DELETE"
    else:
        length = len(textOut.split())
        return [url,"Wired",year,month,day,length,textOut]
df = []
print(scrape("https://www.computerworld.com/article/1638293/microsoft-invested-13b-in-generative-ai-does-copilot-for-windows-show-it-s-a-bad-bet.html")[-1])